In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Upload bibliotecas

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

Upload base de dados

In [3]:
dataset= pd.read_csv('/content/drive/My Drive/Colab Notebooks/acoes.csv')
dataset

dataset.drop(labels = ['Date'], axis = 1, inplace = True)

Normalização da base de dados

In [4]:
dataset_normalizado = dataset.copy()
for i in dataset.columns:
  dataset_normalizado[i] = dataset[i] / dataset[i][0]
dataset_normalizado

,ABEV,ODPV,CCR,PETBR,BB,BOVA
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.981898,1.026681,0.971467,0.914530,0.979249,0.980110
2,1.019351,1.019210,0.985734,0.884615,0.992936,0.985612
3,1.033708,1.034152,1.059783,0.926282,1.036645,1.018832
4,1.032459,1.030950,1.072011,0.986111,1.040177,1.026449
...,...,...,...,...,...,...
1915,0.955056,0.995731,0.920516,3.318376,1.759382,2.245239
1916,0.953184,0.971185,0.920516,3.288462,1.739514,2.234448
1917,0.965668,0.985059,0.938179,3.340812,1.773951,2.289886
1918,0.983770,1.002135,0.930706,3.321581,1.800883,2.303216


In [5]:
dataset_taxa_retorno = (dataset_normalizado / dataset_normalizado.shift(1)) - 1
dataset_taxa_retorno

,ABEV,ODPV,CCR,PETBR,BB,BOVA
0,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.018102,0.026681,-0.028533,-0.085470,-0.020751,-0.019890
2,0.038144,-0.007276,0.014685,-0.032710,0.013977,0.005613
3,0.014084,0.014660,0.075121,0.047101,0.044020,0.033705
4,-0.001208,-0.003096,0.011538,0.064590,0.003407,0.007477
...,...,...,...,...,...,...
1915,-0.005202,0.005388,-0.015977,-0.001928,-0.010184,-0.005995
1916,-0.001961,-0.024652,0.000000,-0.009015,-0.011292,-0.004806
1917,0.013098,0.014286,0.019188,0.015919,0.019797,0.024811
1918,0.018746,0.017335,-0.007965,-0.005756,0.015182,0.005821


In [6]:
dataset_taxa_retorno.fillna(0, inplace=True)
dataset_taxa_retorno.head()

,ABEV,ODPV,CCR,PETBR,BB,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.018102,0.026681,-0.028533,-0.085470,-0.020751,-0.019890
2,0.038144,-0.007276,0.014685,-0.032710,0.013977,0.005613
3,0.014084,0.014660,0.075121,0.047101,0.044020,0.033705
4,-0.001208,-0.003096,0.011538,0.064590,0.003407,0.007477


In [7]:
#Anualizando o retorno

dataset_taxa_retorno.mean() * 246

ABEV     0.033976
ODPV     0.049493
CCR      0.072607
PETBR    0.284057
BB       0.170088
BOVA     0.139135
dtype: float64

Cálculo do parâmetro BETA para um ativo

In [10]:
beta, alpha = np.polyfit(x = dataset_taxa_retorno['BOVA'], y = dataset_taxa_retorno['ABEV'], deg = 1)
print('beta:', beta, 'alpha:', alpha, 'alpha (%):', alpha * 100)

beta: 0.5992445683121849 alpha: -0.00020081194026752789 alpha (%): -0.020081194026752788


In [11]:
figura = px.scatter(dataset_taxa_retorno, x = 'BOVA', y = 'ABEV', title = 'BOVA x ABEV')
figura.add_scatter(x = dataset_taxa_retorno['BOVA'], y = beta * dataset_taxa_retorno['BOVA'] + alpha)
figura.show()

BETA com covariância e variância

In [ ]:
matriz_covariancia = dataset_taxa_retorno.drop(columns = ['ODPV', 'CCR', 'PETBR', 'BB']).cov() * 246
matriz_covariancia

,ABEV,BOVA
ABEV,0.074826,0.039411
BOVA,0.039411,0.065767


In [ ]:
cov_ABEV_bova = matriz_covariancia.iloc[1, 0]
cov_ABEV_bova

0.03941070113207263

In [ ]:
variancia_bova = dataset_taxa_retorno['BOVA'].var() * 246
variancia_bova

0.06576730639891441

In [ ]:
beta_ABEV = cov_ABEV_bova / variancia_bova
beta_ABEV

0.5992445683121844

Cálculo CAPM para uma ação

In [16]:
rm = dataset_taxa_retorno['BOVA'].mean() * 246
rm

0.13913499108701133

In [17]:
taxa_selic_historico = np.array([12.75, 14.25, 12.25, 6.5, 5.0, 2.0])
rf = taxa_selic_historico.mean() / 100
rf

0.08791666666666666

In [ ]:
capm_ABEV = rf + (beta * (rm - rf))
capm_ABEV

0.11860896937360954

Portanto, para o investidor que alocar capital na ação da Ambev ganhará 11% de retorno para ser recompensado pelo risco de mercado de investir nesta ação.

Cálculo do BETA para todas as ações

In [12]:
betas = []
alphas = []
for ativo in dataset_taxa_retorno.columns[0:-1]:
  #print(ativo)
  beta, alpha = np.polyfit(dataset_taxa_retorno['BOVA'], dataset_taxa_retorno[ativo], 1)
  betas.append(beta)
  alphas.append(alpha)

In [13]:
def visualiza_betas_alphas(betas, alphas):
  for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
    #print(i, ativo)
    print(ativo, 'beta:', betas[i], 'alpha:', alphas[i] * 100)

In [14]:
np.array(alphas).mean() * 100

-0.005749600490853083

Calculo do CAPM para o portfólio

In [18]:
capm_empresas = []
for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
  #print(i, ativo)
  capm_empresas.append(rf + (betas[i] * (rm - rf)))

In [19]:
capm_empresas

[0.11860896937360954,
 0.11062227393944903,
 0.1420634850227419,
 0.1633552994500344,
 0.15560085962058803]

In [20]:
def visualiza_capm(capm):
  for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
    print(ativo, 'CAPM:', capm[i] * 100)

In [21]:
visualiza_capm(capm_empresas)

ABEV CAPM: 11.860896937360954
ODPV CAPM: 11.062227393944903
CCR CAPM: 14.20634850227419
PETBR CAPM: 16.33552994500344
BB CAPM: 15.560085962058803


In [22]:
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [23]:
capm_portfolio = np.sum(capm_empresas * pesos) * 100
capm_portfolio

13.80501774812846

Logo, o investidor que alocar capital neste portfólio, deverá receber 13,8% de retorno para ser compensado pelo risco de mercado de investir neste relatório.